In [ ]:
import json
import os
from os.path import isdir
import pandas as pd
import geopandas as gpd
from pyproj import CRS

from osmnx.geocoder import geocode_to_gdf

import ranch
from ranch import sharedstreets
from ranch import Roadway
from ranch import Transit
from ranch import Parameters
from ranch.utils import link_df_to_geojson, point_df_to_geojson
from ranch.logger import RanchLogger

import sharedstreets.dataframe

In [ ]:
counties = pd.read_csv(os.path.join(root_dir, '..', 'data', 'counties.csv'))

In [ ]:
# project directory
root_dir = os.path.join(r"C:\Users\uscd675041\WSP O365\BART-TDLU - Documents\Networks")
# software directory
ranch_dir = os.path.join("c:/apps/cowboy/Ranch")
parameters = Parameters(ranch_base_dir = ranch_dir)

for county_name in list(counties['COUNTY_NAME']):
    alias = county_name.replace(' ', '')
    external_dir = os.path.join(root_dir, alias, 'external')
    interim_dir = os.path.join(root_dir, alias, 'interim')


    # the folder where SharedStreets extractions live
    shst_extract_dir = os.path.join(external_dir, "sharedstreets_extract")
    osm_extract_dir = os.path.join(external_dir, "osmnx_extract")

    for d in [root_dir, external_dir, interim_dir, ranch_dir, shst_extract_dir, osm_extract_dir]:
        if isdir(d):
            print(f"{d} exists!")
            continue

        print(f"creating {d}")
        os.makedirs(d)
    
    if county_name == 'San Francisco': # Because San Francisco is a city and a county / OSMNX gets confused.
        cnty = geocode_to_gdf(f"{county_name}, California, USA").to_crs(epsg=4326)
    else:
        cnty = geocode_to_gdf(f"{county_name} County, California, USA").to_crs(epsg=4326)
    cnty.to_file(os.path.join(root_dir, alias, f'cnty_boundary_{alias}.shp'))

    county_boundary_file = os.path.join(root_dir, alias, f'cnty_boundary_{alias}.shp')
    
    shst = ranch.run_shst_extraction(cnty, output_dir=shst_extract_dir, pylib=True)

    # the OSMNX extraction file is saved to the 'output_dir' argument
    osm_links, osm_nodes = ranch.run_osmnx_extraction(
        input_polygon_file = cnty,
        output_dir = osm_extract_dir
    )

    # build and returns roaday network object from extraction files
    roadway_network = Roadway.create_roadway_network_from_extracts(
        shst.reset_index(),
        osm_links,
        osm_nodes,
        parameters
    )

    RanchLogger.info("write out shape geojson")

    shape_prop = ['id', 'fromIntersectionId', 'toIntersectionId', 'forwardReferenceId', 'backReferenceId']
    shape_geojson = link_df_to_geojson(
        roadway_network.shapes_df, 
        shape_prop
    )

    with open(os.path.join(interim_dir,"step3_shapes.geojson"), "w") as f:
        json.dump(shape_geojson, f)

    RanchLogger.info("write out node geojson")

    node_prop = roadway_network.nodes_df.drop('geometry', axis = 1).columns.tolist()
    node_geojson = point_df_to_geojson(
        roadway_network.nodes_df, 
        node_prop
    )

    with open(os.path.join(interim_dir,"step3_nodes.geojson"), "w") as f:
        json.dump(node_geojson, f)

    RanchLogger.info("write out link geojson")

    link_prop = roadway_network.links_df.drop('geometry', axis = 1).columns.tolist()
    link_geojson = link_df_to_geojson(
        roadway_network.links_df, 
        link_prop
    )

    with open(os.path.join(interim_dir,"step3_links.geojson"), "w") as f:
        json.dump(link_geojson, f)